In [2]:
import scipy.io as scio
import torch
import numpy as np

In [3]:
#----------------subject-subject

In [4]:
def loaddata(datafile_base, num_sub):
    X_train = []
    Y_train = []
    train_cov = []
    for i in range(1,num_sub):
        datafile = datafile_base + str(i)
        data = scio.loadmat(str(datafile))
        MI_train = data['smt'].astype(float)
        MI_train = MI_train.transpose(1,2,0)
        MI_train = np.nan_to_num(MI_train)
        for i in range(100):
            cov1 = np.corrcoef(MI_train[i])
            train_cov.append(cov1)

        x_train = np.array(train_cov)
        y_train = np.transpose(data['y_dec']-1).astype(float)
        y_train = np.nan_to_num(y_train)
    Y_train.append(y_train)
    X_train = torch.from_numpy(np.nan_to_num(np.array(x_train)))
    Y_train = torch.from_numpy(np.array(Y_train))
    X_train = X_train.reshape(-1, 62, 62)
    Y_train = Y_train.reshape(-1, 1).long()
    X_train = X_train.reshape(100*num_sub, 1, 62, 62).float()

    train_dataset = Data.TensorDataset(X_train, Y_train)
    train_loader = Data.DataLoader(
        dataset=train_dataset,      # torch TensorDataset format
        batch_size=32,      # mini batch size
        shuffle=True          
        )
    return train_loader

In [6]:
#-----------------------load data

In [7]:
train_loader = loaddata('', 10) # change the training data root
test_loader = oaddata('', 1) # change the testing data root

In [8]:
from pathlib import Path
import os
import random

import numpy as np
import torch as th
import torch.nn as nn
from torch.utils import data

import nn as nn_spd
from optimizers import MixOptimizer
class RieTransNet(nn.Module):
      def __init__(self):
          super(__class__,self).__init__()
          dim=62
          dim1=58; dim2=54; dim3=50
          classes=2
          self.re=nn_spd.ReEig()
          self.bimap1=nn_spd.BiMap(1,1,dim,dim1)
          self.batchnorm1=nn_spd.BatchNormSPD(dim1)
          self.bimap2=nn_spd.BiMap(1,1,dim1,dim2)
          self.batchnorm2=nn_spd.BatchNormSPD(dim2)
          self.bimap3=nn_spd.BiMap(1,1,dim2,dim3)
          self.batchnorm3=nn_spd.BatchNormSPD(dim3)
          self.logeig=nn_spd.LogEig()
          self.linear=nn.Linear(dim3**2,classes,bias=True)
          self.dropout1 = nn.Dropout(p=0.3)
          self.dropout2 = nn.Dropout(p=0.3)
          self.dropout3 = nn.Dropout(p=0.5)
      def forward(self,x):
          x_spd=self.re(self.batchnorm1(self.bimap1(x)))
          #x_spd=self.dropout1(x_spd)
          x_spd=self.re(self.batchnorm2(self.bimap2(x_spd)))
          #x_spd=self.dropout2(x_spd)
          x_spd=self.batchnorm3(self.bimap3(x_spd))
          x_vec=self.logeig(x_spd).view(x_spd.shape[0],-1)
          x_vec=self.dropout3(x_vec)
          y=self.linear(x_vec.float())
          return y
model=RieTransNet().cuda()

In [ ]:
def BciNet(train, test):

    #main parameters
    threshold_reeig=1e-4 #threshold for ReEig layer
    epochs=100

    #initial validation accuracy
    loss_val,acc_val=[],[]
    y_true,y_pred=[],[]
    gen = test
    model.eval()
    for local_batch, local_labels in gen:
        local_batch = local_batch.float().cuda()
        local_labels = local_labels.long().squeeze().cuda()
        out = model(local_batch)
        l = loss_fn(out, local_labels)
        predicted_labels=out.argmax(1)
        y_true.extend(list(local_labels.cpu().numpy())); y_pred.extend(list(predicted_labels.cpu().numpy()))
        acc,loss=(predicted_labels==local_labels).cpu().numpy().sum()/out.shape[0], l.cpu().data.numpy()
        loss_val.append(loss)
        acc_val.append(acc)
    acc_val = np.asarray(acc_val).mean()
    loss_val = np.asarray(loss_val).mean()
    print('Initial validation accuracy: '+str(100*acc_val)+'%')

    #training loop
    best_acc = 0.00
    for epoch in range(epochs):
        #baselr=0.01
        #opti = MixOptimizer(model.parameters(),lr = baselr * (0.5 ** (epoch // 30)))
        opti = MixOptimizer(model.parameters(),lr = 0.005)

        # train one epoch
        loss_train, acc_train = [], []
        model.train()
        for local_batch, local_labels in train:
            local_batch = local_batch.float().cuda()
            local_labels = local_labels.long().squeeze().cuda()
            opti.zero_grad()
            out = model(local_batch)
            l = loss_fn(out, local_labels)
            acc, loss = (out.argmax(1)==local_labels).cpu().numpy().sum()/out.shape[0], l.cpu().data.numpy()
            loss_train.append(loss)
            acc_train.append(acc)
            l.backward()
            opti.step()
        acc_train = np.asarray(acc_train).mean()
        loss_train = np.asarray(loss_train).mean()
        #print(local_labels)
        #print(out)
        print('Train acc: ' + str(100*acc_train) + '% at epoch ' + str(epoch + 1) + '/' + str(epochs))

        # validation
        loss_val,acc_val=[],[]
        y_true,y_pred=[],[]
        gen = test
        model.eval()
        for local_batch, local_labels in gen:
            local_batch = local_batch.float().cuda()
            local_labels = local_labels.long().squeeze().cuda()
            out = model(local_batch)
            l = loss_fn(out, local_labels)
            predicted_labels=out.argmax(1)
            y_true.extend(list(local_labels.cpu().numpy())); y_pred.extend(list(predicted_labels.cpu().numpy()))
            acc,loss=(predicted_labels==local_labels).cpu().numpy().sum()/out.shape[0], l.cpu().data.numpy()
            loss_val.append(loss)
            acc_val.append(acc)
        acc_val = np.asarray(acc_val).mean()
        loss_val = np.asarray(loss_val).mean()
        #print(local_labels)
        #print(out)
        #print(predicted_labels)
        if acc_val > best_acc:
            best_acc = acc_val
        print('Val acc: ' + str(100*acc_val) + '% at epoch ' + str(epoch + 1) + '/' + str(epochs))
        print(best_acc)


In [19]:
BciNet(train_loader, test_loader)

Initial validation accuracy: 57.03125%
Train acc: 48.4375% at epoch 1/100
Val acc: 42.96875% at epoch 1/100
0.4296875
Train acc: 62.5% at epoch 2/100
Val acc: 56.25% at epoch 2/100
0.5625
Train acc: 56.25% at epoch 3/100
Val acc: 50.0% at epoch 3/100
0.5625
Train acc: 71.875% at epoch 4/100
Val acc: 59.375% at epoch 4/100
0.59375
Train acc: 53.90625% at epoch 5/100
Val acc: 64.84375% at epoch 5/100
0.6484375
Train acc: 65.625% at epoch 6/100
Val acc: 49.21875% at epoch 6/100
0.6484375
Train acc: 64.0625% at epoch 7/100
Val acc: 53.90625% at epoch 7/100
0.6484375
Train acc: 68.75% at epoch 8/100
Val acc: 50.78125% at epoch 8/100
0.6484375
Train acc: 65.625% at epoch 9/100
Val acc: 53.90625% at epoch 9/100
0.6484375
Train acc: 54.6875% at epoch 10/100
Val acc: 67.96875% at epoch 10/100
0.6796875
Train acc: 71.875% at epoch 11/100
Val acc: 54.6875% at epoch 11/100
0.6796875
Train acc: 72.65625% at epoch 12/100
Val acc: 49.21875% at epoch 12/100
0.6796875
Train acc: 85.15625% at epoch 13/1